In [4]:
!pip install transformers
!pip install transformers accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Prepare a system prompt to control the assistant's behavior
system_prompt = "You are a helpful and knowledgeable AI assistant. Answer clearly and concisely."

# Keep track of conversation history
chat_history_ids = None
max_history_tokens = 1024

print("Chat with the AI! Type 'quit' to exit.\n")

while True:  # Arbitrary number of turns
    user_input = input(">> You: ")
    if user_input.lower() in ["quit", "exit"]:
        break

    # Combine system prompt + user input into one message
    prompt = f"{system_prompt}\nUser: {user_input}\nAssistant:"

    # Tokenize input
    input_ids = tokenizer(
        prompt,
        return_tensors="pt"
    ).to(model.device)

    # Append new input to chat history if exists
    bot_input_ids = torch.cat([chat_history_ids, input_ids.input_ids], dim=-1) if chat_history_ids is not None else input_ids.input_ids

    # Keep only the last N tokens for context
    if bot_input_ids.size(-1) > max_history_tokens:
        bot_input_ids = bot_input_ids[:, -max_history_tokens:]

    # Generate a bot response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=bot_input_ids.shape[-1] + 200,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode only new response tokens
    output = tokenizer.decode(
        chat_history_ids[0, bot_input_ids.shape[-1]:],
        skip_special_tokens=True
    )

    print(f"Assistant: {output}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Chat with the AI! Type 'quit' to exit.



>> You:  Capital Of italy?


Assistant: The capital of Italy is Rome, which has been the political center since 1871 when it became part of unified Kingdom in Piedmont-Sardinia under King Victor Emmanuel II from Sabaudian House d'Aosta e Savoia (also known as "House Wittelsbach") after his marriage to Maria Theresa who was heiress by her birthrights over Duchies Parma–Piacenza & Guastalla; however today there exists no official title for this monarch because all previous royal families were abolished during Italian Unification process led mainly through Risorgimento Movement spearheaded primarily among members such us Cavour Giacomo Ricasoli Giuseppe Garibaldi Urbano Rattazzi Massimo D'Azeglio Antonio Canova Niccolò Tommaseo Cesare Battisti Enrico Cialdini Vincenzo Gioberti Giovanni Lanza Ettore Perr


>> You:  Name some data structures


Assistant: Some common types include arrays, linked lists, stacks, queues, trees, graphs, hash tables/maps or dictionaries etc., depending on their functionality requirements like storage efficiency , access speed required . Each one serves different purposes according needs based upon factors being considered while choosing suitable ones given problem domain context where they will be applied within certain algorithmic paradigms adopted towards solving problems efficiently leveraging specific characteristics provided inherently along with particular implementation techniques employed utilizing available resources optimally maximized potential benefits achieved thereby enhancing overall performance outcomes resulting optimal solution space explored effectively covering wide range possible solutions ensuring comprehensive analysis conducted thorough examination reviewing alternatives evaluating pros cons determining best fit approach tailoring customize designs accordingly adaptability 

>> You:  exit
